In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from google.cloud import bigquery

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [20]:
import bq_helper
# create a helper object for our bigquery dataset
covid_19 = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "covid19_jhu_csse")

covid_19.table_schema("confirmed_cases")


Using Kaggle's public dataset BigQuery integration.


,name,type,mode,description
0,province_state,STRING,NULLABLE,None
1,country_region,STRING,NULLABLE,None
2,latitude,FLOAT,NULLABLE,None
3,longitude,FLOAT,NULLABLE,None
4,location_geom,GEOGRAPHY,NULLABLE,None
...,...,...,...,...
235,_9_8_20,INTEGER,NULLABLE,None
236,_9_9_20,INTEGER,NULLABLE,None
237,_9_10_20,INTEGER,NULLABLE,None
238,_9_11_20,INTEGER,NULLABLE,None


In [21]:
covid_19.list_tables()

['confirmed_cases', 'deaths', 'recovered_cases', 'summary']

In [227]:
import datetime
def date_generator(start_date, end_date):
    start_date = start_date[1:]
    end_date = end_date[1:]
    
    new_start_date = np.asarray(start_date.split('_'))
    new_end_date = np.asarray(end_date.split('_'))
    
    dt = datetime.date(int(new_start_date[2]), int(new_start_date[0]), int(new_start_date[1]))
    end = datetime.date(int(new_end_date[2]), int(new_end_date[0]), int(new_end_date[1]))
    step = datetime.timedelta(days = 1)
    

    result = []

    while dt < end:
        newtime = dt.strftime('_%m_%d_%Y').replace('_0','_')
        result.append(newtime)
        dt += step
    return result

def real_dates(start_date,end_date):
    start_date = start_date[1:]
    end_date = end_date[1:]
    
    new_start_date = np.asarray(start_date.split('_'))
    new_end_date = np.asarray(end_date.split('_'))
    
    dt = datetime.date(int(new_start_date[2]), int(new_start_date[0]), int(new_start_date[1]))
    end = datetime.date(int(new_end_date[2]), int(new_end_date[0]), int(new_end_date[1]))
    step = datetime.timedelta(days = 1)
    
    real_dates = []
    
    while dt < end:
        real_dates.append(dt.strftime('%B %d, %Y'))
        dt += step
    return real_dates

In [334]:
def country_finder(country, start_date, end_date):

    dates = date_generator(start_date,end_date)
    string_dates = ', '.join(dates)
    
#########################################################################################################
#            
#                                       CONFIRMED CASES
#
#########################################################################################################

    cc_country_query = """
                    WITH relevant_country AS
                    (
                    SELECT *
                    FROM `bigquery-public-data.covid19_jhu_csse.confirmed_cases`
                    WHERE country_region = '{}'
                    )
                    SELECT {} AS Total_Cases, {} AS start_cases, {}
                    FROM relevant_country
                    """.format(country, end_date, start_date, string_dates)
    
    results_cc = covid_19.query_to_pandas_safe(cc_country_query, max_gb_scanned=0.1)
    
    summed_cc = results_cc.sum(axis=0)

    x = np.asarray(summed_cc)
    
    cc_start = x[1]
    cc_total = x[0]
    
#########################################################################################################
#            
#                                       DEATHS
#
#########################################################################################################

    death_country_query = """
                    WITH relevant_country AS
                    (
                    SELECT *
                    FROM `bigquery-public-data.covid19_jhu_csse.deaths`
                    WHERE country_region = '{}'
                    )
                    SELECT {} AS Total_Cases, {} AS start_cases, {}
                    FROM relevant_country
                    """.format(country, end_date, start_date, string_dates)
    
    results_death = covid_19.query_to_pandas_safe(death_country_query, max_gb_scanned=0.1)
    
    summed_death = results_death.sum(axis=0)

    y = np.asarray(summed_death)
    
    death_start = y[1]
    death_total = y[0]


#########################################################################################################
#            
#                                       RECOVERED
#
#########################################################################################################


    rec_country_query = """
                    WITH relevant_country AS
                    (
                    SELECT *
                    FROM `bigquery-public-data.covid19_jhu_csse.recovered_cases`
                    WHERE country_region = '{}'
                    )
                    SELECT {} AS Total_Cases, {} AS start_cases, {}
                    FROM relevant_country
                    """.format(country, end_date, start_date, string_dates)
    
    results_rec = covid_19.query_to_pandas_safe(rec_country_query, max_gb_scanned=0.1)
    
    summed_rec = results_rec.sum(axis=0)

    z = np.asarray(summed_rec)
    
    rec_start = z[1]
    rec_total = z[0]
    
    
#########################################################################################################
#            
#                                       PLOTS
#
#########################################################################################################

    
    real_dates_for_plotting = real_dates(start_date,end_date)
    
    length = len(dates)
    plot_dates = []
    indices = []
    a = int(length/8)
    for i in range(0,8):
        date = real_dates_for_plotting[a*i]
        plot_dates.append(date)
        indices.append(a*i)
    
    
    plt.figure(figsize = [15,10])
    plt.plot(range(0,length),x[2:], label="Confirmed Cases")
    plt.plot(range(0,length),y[2:], label="Deaths")
    plt.plot(range(0,length),z[2:], label="Recovered Cases")
    plt.xlabel('Time', size = 25)
    plt.ylabel('No. of Cases', size = 25)
    plt.xticks((indices), (plot_dates), size = 15,rotation=45)
    plt.yticks(size = 15)
    plt.ylim(0,cc_total+0.05*cc_total)
    plt.legend()
    plt.show()


In [335]:
qu = """
    SELECT country_region, COUNT(1) AS num_occ
    FROM `bigquery-public-data.covid19_jhu_csse.recovered_cases`
    GROUP BY country_region
    ORDER BY num_occ DESC
    """
    
results = covid_19.query_to_pandas_safe(qu, max_gb_scanned=0.1)
numpy_results = np.asarray(results)
nummy = np.sort(numpy_results[:,0])
nummy = np.insert(nummy, 0, 'Global')

In [449]:
# Global function
import plotly.graph_objects as go

import pandas as pd
import plotly.express as px
import pycountry

def glob_fn(start_date,end_date):
    
    dates = date_generator(start_date,end_date)
    string_dates = ', '.join(dates)

    ############################################
    # Global confirmed cases
    ############################################
    cc_glob_query = """
                    SELECT {} AS Total_Cases, {} AS start_cases, {}
                    FROM `bigquery-public-data.covid19_jhu_csse.confirmed_cases`
                    """.format(end_date, start_date, string_dates)
    
    results_glob_cc = covid_19.query_to_pandas_safe(cc_glob_query, max_gb_scanned=0.1)
    
    summed_glob_cc = results_glob_cc.sum(axis=0)

    glob_x = np.asarray(summed_glob_cc)
    
    glob_cc_start = glob_x[1]
    glob_cc_tot = glob_x[0]

    ############################################
    # Global deaths
    ############################################
    death_glob_query = """
                    SELECT {} AS Total_Cases, {} AS start_cases, {}
                    FROM `bigquery-public-data.covid19_jhu_csse.deaths`
                    """.format(end_date, start_date, string_dates)
    
    results_glob_death = covid_19.query_to_pandas_safe(death_glob_query, max_gb_scanned=0.1)
    
    summed_glob_death = results_glob_death.sum(axis=0)

    glob_y = np.asarray(summed_glob_death)
    
    glob_death_start = glob_y[1]
    glob_death_tot = glob_y[0]
    
    ############################################
    # Global recovered
    ############################################
    rec_glob_query = """
                    SELECT {} AS Total_Cases, {} AS start_cases, {}
                    FROM `bigquery-public-data.covid19_jhu_csse.recovered_cases`
                    """.format(end_date, start_date, string_dates)
    
    results_glob_rec = covid_19.query_to_pandas_safe(rec_glob_query, max_gb_scanned=0.1)
    
    summed_glob_rec = results_glob_rec.sum(axis=0)

    glob_z = np.asarray(summed_glob_rec)
    
    glob_rec_start = glob_z[1]
    glob_rec_tot = glob_z[0]
  

    ############################################
    # Plots
    ############################################
    
    real_dates_for_plotting = real_dates(start_date,end_date)
    
    length = len(dates)
    plot_dates = []
    indices = []
    a = int(length/8)
    for i in range(0,8):
        date = real_dates_for_plotting[a*i]
        plot_dates.append(date)
        indices.append(a*i)
    
    
    plt.figure(figsize = [15,10])
    plt.plot(range(0,length),glob_x[2:], label="Confirmed Cases")
    plt.plot(range(0,length),glob_y[2:], label="Deaths")
    plt.plot(range(0,length),glob_z[2:], label="Recovered Cases")
    plt.xlabel('Time', size = 25)
    plt.ylabel('No. of Cases', size = 25)
    plt.xticks((indices), (plot_dates), size = 15,rotation=45)
    plt.yticks(size = 15)
    plt.ylim(0,glob_cc_tot+0.05*glob_cc_tot)
    plt.legend()
    plt.show()
    

    ############################################
    # Heat map
    ############################################
    
    heat_query = """
                    SELECT country_region, {} AS Total_Cases, latitude
                    FROM `bigquery-public-data.covid19_jhu_csse.confirmed_cases`
                    WHERE province_state is NULL
                    """.format(end_date)
    
    results_heat = covid_19.query_to_pandas_safe(heat_query, max_gb_scanned=0.1)
    
    array = np.asarray(results_heat)
    
    for i in range(len(array[:,0])):
        country_in_q = array[i,0]
        if False:
            pass
        else:
            try:
                t = pycountry.countries.get(name='{}'.format(country_in_q))
                if t is None:
                    t = pycountry.countries.search_fuzzy(country_in_q)
                    iso = t[0].alpha_3
                    array[i,2] = iso
                else:
                    iso = t.alpha_3
                    array[i,2] = iso
            except (LookupError):
                pass
        array[i,1] = float(array[i,1])
    
    df = pd.DataFrame(data=array, columns=["country", "Cases", "ISO"])
    df.astype({'Cases': 'int32'}).dtypes
    
    fig = px.choropleth(df, locations="ISO",
                    color="Cases",
                    hover_name="country",
                    color_continuous_scale=px.colors.sequential.Plasma)
    fig.show()


In [450]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from datetime import date


today = date.today()
the_year = int(today.strftime("%Y"))
the_month = int(today.strftime("%m"))
the_date = int(today.strftime("%d"))

st_date = "{}-{}-{}".format(the_year,the_month,the_date)

years_available = []

for i in range(2020,the_year+1):
    years_available.append(i)
    
months_available = []

for j in range(1,the_month):
    months_available.append(j)
    
days_available = []
for k in range(1, the_date):
    days_available.append(k)

all_months = [('January'),('February'),('March'),('April'),('May'),('June'),
              ('July'),('August'),('September'),('October'),('November'),('December')]

all_dates = []
for m in range(1,32):
    all_dates.append(m)
    
choose_month1 = widgets.Dropdown(
    options=all_months,
    value='January',
    disabled=False,
)

choose_year1 = widgets.Dropdown(
    options=years_available,
    value=2020,
    disabled=False,
)

choose_date1 = widgets.Dropdown(
    options=all_dates,
    value=25,
    disabled=False,
)

choose_month2 = widgets.Dropdown(
    options=all_months,
    value='August',
    disabled=False,
)

choose_year2 = widgets.Dropdown(
    options=years_available,
    value=2020,
    disabled=False,
)

choose_date2 = widgets.Dropdown(
    options=all_dates,
    value=1,
    disabled=False,
)

ctry = widgets.Dropdown(
    options=nummy,
    value='Global',
    description='Choose country:',
    disabled=False,
)

def function(Country, Year1, Month1, Day1, Year2, Month2, Day2):
    
    for u in range(0,12):
        if Month1 == all_months[u]:
            Month1 = u+1
    for y in range(0,12):
        if Month2 == all_months[y]:
            Month2 = y+1
    
    first_date = "{}-{}-{}".format(Year1, Month1, Day1)
    second_date = "{}-{}-{}".format(Year2, Month2, Day2)
    
    if first_date > second_date or second_date > st_date:
        print("You shall not pass!")
    else:
        yearsies1 = str(Year1)[2:]
        datesies1 = Day1
        monthies1 = Month1
            
        yearsies2 = str(Year2)[2:]
        datesies2 = Day2
        monthies2 = Month2
            
        d1 = "_{}_{}_{}".format(monthies1, datesies1, yearsies1)
        d2 = "_{}_{}_{}".format(monthies2, datesies2, yearsies2)
        if Country == 'Global':
            glob_fn(d1, d2)
        else:
            country_finder(Country, d1, d2)
            
            


Select = widgets.interactive(function, Country=ctry, Year1=choose_year1, Month1=choose_month1, Day1=choose_date1,
                             Year2=choose_year2, Month2=choose_month2, Day2=choose_date2)


display(Select)

interactive(children=(Dropdown(description='Choose country:', options=('Global', 'Afghanistan', 'Albania', 'Al…